# [ERG-190C] Homework 8: Resampling & Cross-Validation

---

This assignment will take a step back from building models (hw5, hw6, hw7) and begin exploring resampling techniques to assess models and/or estimators. This process is important to avoid overfitting, and to learn more about the model you have created.

---

<img src='imgs/cv.png' width="50%" height="50%"></img>

### Topics Covered

ISLR Chapter 5

- Dangers of Overfitting
- K-Fold Cross Validation

### Table of Contents

[Introduction](#intro)<br>
1 - [Overfitting](#1)<br>
2 - [Cross Validation](#2)<br> 
[Conclusion](#conclusion)<br>

**Dependencies:**

In [ ]:
import numpy as np
import pandas as pd
import utils

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set_context("talk")
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

np.random.seed(1)

In [ ]:
# Run this block. You will need these libraries for one of the questions.
!pip install plotly
!pip install cufflinks

## Introduction <a name='intro'></a>

Welcome to Homework 8! In this homework, you will learn to utilize two resampling methods: cross-validation, and bootstrapping. You've worked hard in the past few homeworks (hw5, hw6, hw7) to understand how you can develop your estimation models through linear regression and feature engineering to best represent your target variables and to lower your training error rates.

However, **how will our model perform on data it has not seen yet?** Now, you'll shift your focus away from fine-tuning your model, and begin to explore ways to _validate_ your model's performance by withholding some of the data given to you initially in order to generalize your estimator.

**Learning objectives:**

1. Recognizing the pitfalls of overfitting, when it will occur.
1. Circumventing the issue of overfitting the model by preparing the dataset for cross-validation. 
1. Apply k-fold cross-validation to a training set and analyze how it estimates the error rate for future data.

## Section 1: Overfitting <a name='1'></a>

In this class, we have been using mean squared error (MSE) as a way of quantifying error from our models, which is in the form:

$$
\text{MSE} = \frac{1}{n}\sum\limits_{i=1}^{n}\Big(y_i-\hat{f}(x_i)\Big)^2
$$

where $y_i$ is our target $y$ and $\hat{f}(x_i)$ is our estimator's best guess of the value $y$.

Ideally, our goal is to lower the MSE in our estimator model. However, we will soon learn when it is bad to minimize any further. As a basic example, we will try and choose a model with two parameters to fit linear data (with some noise added). By now, you should have a good idea of the variables that go into a model, and how to fine-tune them to decrease MSE.

We are going to create some pseudo-random data for the purposes of this homework. Read the block below before moving on and try to recite to yourself what each variable will represent. The answers for three of the variables are right below.

In [ ]:
# Run this block
n = 50
sigma_2 = 100
noise = np.random.normal(scale=np.sqrt(sigma_2), size=n)
X = np.random.uniform(-10, 10, n)
Y3 = 100 + 20. * X  - 2 * X**2 + noise
Y = Y3

* `n` is the number of datapoints in our space.
* `X` is the x-values of our data
* `Y` is the non-biased estimator of our data. The model will roughly take its shape, but with a bit of added noise.

**Question 1.1:** Explain in three or less sentences what this part of the code represents about the data we are creating:

```python
np.random.normal(scale=np.sqrt(sigma_2), size=n)
```

*Hint:* This [link](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.normal.html) may help you.

**Answer**: YOUR ANSWER HERE

** EXAMPLE ANSWER **
* `sigma_2` will be the variance. 
* `noise` is a random number taken from the Gaussian distribution within the range of the square root of the variance, or the standard deviation.

We now have a dataset with a known linear covariance, but it would be easier if we could visualize it. Luckily, we already know how to do that. Let's warm up our skills with visualization.

<br>
**Question 1.2:** Create a scatterplot of the values of `X` and `Y3` using the `matplotlib` library. Change the size of the dots to 15. Be sure to give your plot a title.

In [ ]:
# YOUR CODE HERE
...
plt.title(...);

<br>
**Question 1.3:** Here's the second-order formula for a model with three parameters. Notice that this is in the format of the estimator we used to create our data.

$$ \Large
y_i = \beta_0 + \beta_1 x_i + \beta_2 x_i^2
$$

In the block below, describe in one sentence what each variable represents. The first one is done as an example:

**ANSWER**:

* $\large y_i$ [__Example Answer__]: This is the estimated value of the true y based on our model's parameters.
* $\large\beta_0$: YOUR ANSWER HERE
* $\large\beta_1$: YOUR ANSWER HERE
* $\large\beta_2$: YOUR ANSWER HERE
* $\large x_i$: YOUR ANSWER HERE

Now that we have a good grasp of what our dataset represents, we are going to try and find a model that best estimates unknown values of $x$. To do this, we need to split our data into train and test sets.

<br>
**Question 1.4:** Below, take our data `X`, `Y` and create new data arrays for our new training and test sets. There should be 60% data inside of our training set, and 40% data in our test set.

Then, plot the training and testing data on the same scatter plot. Make the training points blue dots, and the test points red diamonds. Both sets should have marker size 20.

*Hint: Use the `sklearn.model_selection` library. There should be an extremely helpful function you can use for this purpose.*

In [ ]:
# YOUR CODE HERE
X_train, y_train, X_test, y_test = ...
assert [np.size(X_train), np.size(y_train)] == [25, 25]

plt.scatter(...)
...

It's time to fit our first model! What if we weren't sure what the relationship correlation looked like? Normally, you'd start with the most simple model and improve from there. So, that is exactly what we will do - we will create a feature matrix $\Phi$ that will fit a first-order model to our data.

$$ \large
\Phi_d(x) = \left[x, x^2, \ldots, x^d \right]
$$

$$ \large
\Phi_1(x) = \left[x \right]
$$

Depending on how many features (degrees) we are adding to our model, we account for it by the following function:

In [ ]:
def poly_phi(k):
    return lambda X: np.array([X ** i for i in range(1, k+1)]).T

We are focusing on just one feature for this question, so let $k=1$, and the implicit $X = X_{train}$.

<br>
**Question 1.5:** Our new X training data will be the transformed $\Phi_1$ matrix. Use the `LinearRegression()` object from `sklearn` to fit the and train the model, then plot the resulting estimator overlayed on the original training data based on the model's predictions for y at the values of `X_plt` (This is a randomized set of x-values within a small range to make plotting our model easier). The scatter plot should still contain the distinctions between the training and test sets.

*Hint: Here's the documentation for the `LinearRegression` object again: [docs](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)*

*Hint: In order to call `model.predict()` correctly, you'll need to transform the `X_plt` so that it is formatted as a $\Phi$ feature matrix with just first-order.*

*Hint: Your graph should look similar to this:*

<img src='imgs/deg1.png' width="50%" height="50%"></img>

In [ ]:
# YOUR CODE HERE
k = 1
phi_1 = poly_phi(k)(X_train)
# Make the x values where plot points will be generated
X_plt = np.linspace(np.min(X)-1, np.max(X)+1, 200)

model = ...
...
y_pred = model.predict(...)

plt.scatter(...)
...
plt.title(...);

This is a good start. If we notice the vertical distances of the train data from the best fit line, we'll still get a very high mean squared error value.


What we have just illustrated is an example of **underfitting**. The model is not quite capturing the underlying trend of the data. So, how do we fix this? The simple answer seems to be to fit as many features in as possible, right?

Not so fast. Next, you'll see why too many features can be especially bad.

It's easy to think that we'll need many features so we can estimate the "perfect model". We're going to show why that is the wrong way to approach this problem. Now, we will create a 24-order polynomial estimator. We expect the training accuracy to be extremely high! Is our expectation correct?

<br>
**Question 1.6:** Repeat the steps you followed for Question 1.5, only this time, we will create a feature matrix of 24 variables. Again, make a visualization of your findings.

*Hint: *

$$ \large
\Phi_{24}(x) = \left[x^1,x^2,...,x^{24} \right]
$$

In [ ]:
# YOUR CODE HERE
k = ...
... = poly_phi(k)(X_train)
# Make the x values where plot points will be generated
X_plt = np.linspace(np.min(X)-1, np.max(X)+1, 200)

model = ...
...
y_pred = model.predict(...)

# Visualization
plt.scatter(...)
... # hint: plt.ylim(ymin=..., ymax=...)
plt.title(...);

Was our initial assumption correct (Recall that our assumption was that the training error, or MSE, would go down with MANY features)?

The answer is a resounding NO. We have show that there is such a case such that we have added *too many* features - this is called **overfitting.** We attempted to use 24 features to improve accuracy, but as a result, when new data was added in, the model had no idea how to create a best fit-line. Take a look at how the model tried to fit the data between $x=[5,10]$ above. Intuitively, we know that this model is outrageous.

<br>
**Question 1.7:** We now have seen the model fitting with 1st-order and 24-order polynomials. In less than three sentences, what can we now say about how many _features_ we should use to fit this data? 

**ANSWER**: YOUR ANSWER HERE.

<br>
Good job! You are almost finished with Question 1. To deepen your understanding, we've written some code that will output an interactive graph that you can play with to show when a function is overfitted or underfitted. Just run the blocks below and answer the question at the end.

In [ ]:
basis_sizes = [1, 2, 5, 8, 16, 24, 32]

In [ ]:
Phis_train = { k: poly_phi(k)(X_train) for k in basis_sizes }
from sklearn import linear_model
models = {}
for k in Phis_train:
    model = LinearRegression()
    model.fit(Phis_train[k], y_train)
    models[k] = model

In [ ]:
utils.interact_plot(basis_sizes, models, poly_phi, X, Y)

**Question 1.8:** Choose your estimation for the best fit on the training and test data. Which degree polynomial is it? Give reasoning for your answer.

**ANSWER**: YOUR ANSWER HERE

----

## Section 2: Validation<a name='2'></a>

You have now estimated the best fit for the data we created. Let's think about the steps we took to get to a successful answer: 

1. First, we started with a simple first-order polynomial to have a baseline to our training error.

2. We noticed from our graphing that it was insufficient, so we continued to add more features...

3. ... until we reached a point in which both the training data and the test data would both be well-fit with the number of features we have chosen.

However, you have been taught in this class to not touch the testing data until we have a model that can generalize to unseen data. To run the `.predict()` function on the test data and tuning our parameters would in turn convert our test set into a training set! However, we still want a way to _validate_ the model so that we can decide on an ideal regression. So what is the best way to do this? We'll explore one way to do this: **k-fold cross validation**.

### Cross Validation

<img src='imgs/cv.png' width="50%" height="50%"></img>

Let's discuss what we see in the figure above. We are given a dataset to work with, and do our usual train-test split. We want to set aside the test data until we are absolutely ready to test the model we have created. However, in order to make sure that the model works as planned on data it is not 100% familiar with, we use cross-validation. In the image above, it is split into 4-folds. Essentially, each V that you see above represents a quarter of the training data set. This then acts as the test set, and the model is fitted on the remaining 3/4 of the train. This process is repeated three more times on different subsets, and the average MSE indicates the total validity of the algorithm on the "unseen" data.

<br>
**Question 2.1:** Let's practice splitting training data into k-folds for validation purposes. Split the `X_train` array from before into 3 folds, shuffling before we add the batches, with a random state of 7. For each fold, print the indices of the Train and Validation sets onto the console.

In [ ]:
# YOUR CODE HERE
... 

You should have noticed that each validation set is disjoint from the others, meaning that none of the validation sets share common items even though they are shuffled pretty well. Let's think back to what we were doing in Problem #1. We were trying to figure out which model best exemplified a good fit to both the training and test sets. However, we won't always be able to rely on just visualization for all cases. This is where k-fold and what you learned comes in!

<br>
**Question 2.2:** Let's do a little review first before you start implementing the k-fold. Recall the formula for MSE (look back to the top of the notebook if you don't remember what it looks like). Implement the function below to calculate the mean squared error between an array of predicted and true y-values.

In [ ]:
# YOUR CODE HERE
def mse(y_pred, y_true):
    return ...

Recall back to the start of this homework when you made a list of trained models at different numbers for features. We can use k-fold to find out which one has the lowest mean squared error when predicting y-values.

Here's the `basis_sizes` variable for reference:

In [ ]:
basis_sizes

**Question 2.3:** Complete the function `k_fold_lr` below. We are seeking to create a new k-fold cross validation pipeline with 3-folds, shuffling, and a random state of 7. Remember, we are only splitting the training set into k-folds. Then, find the average mse for each validation split. The function should return the average of the MSEs for each number of parameters as a dictionary, with each key as a number representing the parameter size, and the value being the 3-fold average of the MSEs. The dictionary should be saved in a variable called `average_mses`.

In [ ]:
# YOUR CODE HERE
def mse_k_fold_lr(n, basis_sizes, poly_phi, mse, X_train, y_train):
    '''This function will print the basis size that gives the smallest MSE.
       Returns the dictionary of average MSEs (from 3-fold cross validation) across the specified basis sizes
    '''
    
    # 1. K-Fold where k = n
    kf = ...
    
    # 2. Save the MSEs of each split.
    mses = {}
    for train_index, val_index in kf.split(X_train):
        # 2.1 Separate each array into respective variables
        X_fold_train = ...
        y_fold_train = ...
        X_fold_val = ...
        y_fold_val = ...
        
        # 2.2 Use poly_phi(...)(...) to create a feature matrix for each basis size.
        Phis_fold_train = { k: poly_phi(k)(X_fold_train) for k in basis_sizes }
        
        # 2.3 Each basis size will receive its own LinearRegression() model.
        for phi_fold_k in Phis_fold_train:
            # 2.3.1 Create the model
            model = ...
            
            # 2.3.2 Fit the data on the newly created feature matrix at phi_fold_k and the y_fold_train
            ...
            
            # 2.3.3 Predict y-values for the validation set transformed into a feature matrix
            y_pred = ...
            
            # Saving each mse between y_pred and y_fold_val at their respective basis size value.
            if phi_fold_k in mses:
                mses[phi_fold_k].append(mse(y_pred, y_fold_val))
            else:
                mses[phi_fold_k] = [mse(y_pred, y_fold_val)]
    
    # 3. Now, find the average of the MSEs at each k-basis size. This should give you
    # .  the average of MSEs on 3 different val sets per each basis size.
    average_mses = {}
    for k in basis_sizes:
        ...
    
    # 4. Find the index of the minimum average MSE
    min_mse_index = ...
    
    print("Minimum MSE Parameters:", min_mse_index, \
          "MSE of {} Parameters:".format(min_mse_index), average_mses[min_mse_index])
    return ...


average_mses = mse_k_fold_lr(3, basis_sizes, poly_phi, mse, X_train, y_train)
average_mses

To check that this is the true minimum MSE for all of the possible number of parameters, we can visualize this different MSE values for each parameter.

**Question 2.4:** Use the `matplotlib` or `seaborn` libraries to visualize the average *ROOT* mean squared error (from cross-validating) and to confirm your answer to Question 2.3. Think carefully about which type of plot would be most appropriate for this visualization, and what variables we are plotting.

In [ ]:
# YOUR CODE HERE
...

## Conclusion <a name='conclusion'></a>

Congratulations, you've reached the end! By now, you should have a solid understanding of what it means to overfit a graph, and how to test whether you have done so. Incidentally, we also learned a very important resampling technique: cross validation.

----

## Submission

Before you submit, click **Kernel** --> **Restart & Clear Output**. Then, click **Cell** --> **Run All**. Then, go to the toolbar and click **File** -> **Download as** -> **.html** and submit the file through bCourses.

----

## Bibliography


http://ds100.org/

---
Notebook developed by: Alex Nakagawa

Data Science Modules: http://data.berkeley.edu/education/modules